<h1> This notebook shows how to convert a workflow instance into a QUBO </h1>

<div class="alert alert-block alert-info">
    <b> Create a workflow instance. </b>
</div>

In [1]:
from QHyper.problems.workflow_scheduling import (
    Workflow,
    WorkflowSchedulingProblem,
)

In [2]:
tasks_file =  "workflows_data/workflows/4_tasks_3_machines_2_paths.json"
machines_file = "workflows_data/machines/machines_for_3_tasks_3_machines_1_path.json"
deadline = 20

In [3]:
workflow = Workflow(tasks_file, machines_file, deadline)
wsp = WorkflowSchedulingProblem(workflow)

In [4]:
wsp.workflow.time_matrix

,MachineA,MachineB,MachineC
Task1,3.0,1.0,2.0
Task2,6.0,2.0,4.0
Task3,12.0,4.0,8.0
Task4,24.0,8.0,16.0


In [5]:
wsp.workflow.cost_matrix

,MachineA,MachineB,MachineC
Task1,3.0,4.0,4.0
Task2,6.0,8.0,8.0
Task3,12.0,16.0,16.0
Task4,24.0,32.0,32.0


In [6]:
print(
    f"The objective function as a string is:\n {wsp.objective_function.polynomial}"
)

The objective function as a string is:
 3.0*x0 + 4.0*x1 + 32.0*x10 + 32.0*x11 + 4.0*x2 + 6.0*x3 + 8.0*x4 + 8.0*x5 + 12.0*x6 + 16.0*x7 + 16.0*x8 + 24.0*x9


In [7]:
print(
    f"The objective function as a dict is:\n {wsp.objective_function.as_dict()}"
)

The objective function as a dict is:
 {('x0',): 3.0, ('x1',): 4.0, ('x10',): 32.0, ('x11',): 32.0, ('x2',): 4.0, ('x3',): 6.0, ('x4',): 8.0, ('x5',): 8.0, ('x6',): 12.0, ('x7',): 16.0, ('x8',): 16.0, ('x9',): 24.0}


In [8]:
print("Constraints as dicts:")
for constraint in wsp.constraints:
    print(constraint.as_dict())

Constraints as dicts:
{('x0',): 1, ('x1',): 1, ('x2',): 1, (): -1}
{('x3',): 1, ('x4',): 1, ('x5',): 1, (): -1}
{('x6',): 1, ('x7',): 1, ('x8',): 1, (): -1}
{('x10',): 1, ('x11',): 1, ('x9',): 1, (): -1}
{('s0',): 1, ('s1',): 2, ('s2',): 4, ('x0',): 3.0, ('x1',): 1.0, ('x10',): 8.0, ('x11',): 16.0, ('x2',): 2.0, ('x3',): 6.0, ('x4',): 2.0, ('x5',): 4.0, ('x9',): 24.0, (): -20}
{('s0',): 1, ('s1',): 2, ('s2',): 4, ('x0',): 3.0, ('x1',): 1.0, ('x10',): 8.0, ('x11',): 16.0, ('x2',): 2.0, ('x6',): 12.0, ('x7',): 4.0, ('x8',): 8.0, ('x9',): 24.0, (): -20}


In [9]:
wsp.get_deadlines()  # those are the fastest and the slowest execution times

(13.0, 39.0)

<div class="alert alert-block alert-info">
    <b> Convert the workflow to QUBO form. </b>
</div>

In [10]:
from QHyper.solvers.converter import Converter

In [11]:
converter = Converter()

The converter returns the QUBO as a dict, and numerical offset. We partially use [this](https://test-projecttemplate-dimod.readthedocs.io/en/latest/reference/bqm/generated/dimod.BinaryQuadraticModel.to_qubo.html) method.

In [12]:
wsp_qubo, offset = converter.to_qubo(wsp)

In [13]:
wsp_qubo

{('x1', 'x0'): 140.0,
 ('x10', 'x0'): 960.0,
 ('x10', 'x1'): 320.0,
 ('x11', 'x0'): 1920.0,
 ('x11', 'x1'): 640.0,
 ('x11', 'x10'): 5140.0,
 ('x2', 'x0'): 260.0,
 ('x2', 'x1'): 100.0,
 ('x2', 'x10'): 640.0,
 ('x2', 'x11'): 1280.0,
 ('x3', 'x0'): 360.0,
 ('x3', 'x1'): 120.0,
 ('x3', 'x10'): 960.0,
 ('x3', 'x11'): 1920.0,
 ('x3', 'x2'): 240.0,
 ('x4', 'x0'): 120.0,
 ('x4', 'x1'): 40.0,
 ('x4', 'x10'): 320.0,
 ('x4', 'x11'): 640.0,
 ('x4', 'x2'): 80.0,
 ('x4', 'x3'): 260.0,
 ('x5', 'x0'): 240.0,
 ('x5', 'x1'): 80.0,
 ('x5', 'x10'): 640.0,
 ('x5', 'x11'): 1280.0,
 ('x5', 'x2'): 160.0,
 ('x5', 'x3'): 500.0,
 ('x5', 'x4'): 180.0,
 ('x6', 'x0'): 720.0,
 ('x6', 'x1'): 240.0,
 ('x6', 'x10'): 1920.0,
 ('x6', 'x11'): 3840.0,
 ('x6', 'x2'): 480.0,
 ('x7', 'x0'): 240.0,
 ('x7', 'x1'): 80.0,
 ('x7', 'x10'): 640.0,
 ('x7', 'x11'): 1280.0,
 ('x7', 'x2'): 160.0,
 ('x7', 'x6'): 980.0,
 ('x8', 'x0'): 480.0,
 ('x8', 'x1'): 160.0,
 ('x8', 'x10'): 1280.0,
 ('x8', 'x11'): 2560.0,
 ('x8', 'x2'): 320.0,
 ('x8'

In [14]:
offset

8040.0

-----------------


<h3> This part shows how to use solvers from QHyper </h3>

<div class="alert alert-block alert-info">
    <b> Solve the Workflow Schedluling Problem with the CQM solver. </b>
</div>

In [15]:
from QHyper.solvers.cqm import CQM

In [16]:
cqm_problem = CQM(problem=wsp, time=5)

In [17]:
solution = cqm_problem.solve()
print(solution)

{'s0': 1.0, 's1': 0.0, 's2': 1.0, 'x0': 1.0, 'x1': 0.0, 'x10': 1.0, 'x11': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 1.0, 'x6': 0.0, 'x7': 1.0, 'x8': 0.0, 'x9': 0.0}


In [18]:
wsp.decode_solution(solution)

{'Task1': 'MachineA',
 'Task4': 'MachineB',
 'Task2': 'MachineC',
 'Task3': 'MachineB'}

<div class="alert alert-block alert-info">
    <b> Solve the Workflow Schedluling Problem with the Gurobi solver. </b>
</div>

In [19]:
from QHyper.solvers.gurobi import Gurobi

In [20]:
gurobi = Gurobi(problem=wsp)

In [22]:
solution_gurobi = gurobi.solve()

TypeError: Gurobi.solve() missing 1 required positional argument: 'params_inits'

In [23]:
solution_gurobi

NameError: name 'solution_gurobi' is not defined

In [24]:
wsp.decode_solution(solution_gurobi)

NameError: name 'solution_gurobi' is not defined